# UNet with Resnet 50 Backbone

Train Dataset Size: 

Val Dataset Size: 

Test Dataset Size:

In [1]:
# %pip install torch -q
# %pip install opencv-python -q
# %pip install pycocotools -q
# %pip install timm==0.6.12 -q
# %pip install ipdb -q

In [10]:
from backbones_unet.model.unet import Unet
from backbones_unet.utils.dataset import SemanticSegmentationDataset
from backbones_unet.model.losses import DiceLoss
from backbones_unet.utils.trainer import Trainer
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from convert_coco_ann_to_mask import convert_coco_to_mask
from torchvision.datasets import ImageFolder

import torchvision
import torch
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [11]:
# Test Installation
random_tensor = torch.rand((1, 3, 64, 64))
model = Unet(in_channels=3, num_classes=1) # if no backbone specified, will default to Resnet50
print(model.predict(random_tensor))
# summary(model, random_tensor)

tensor([[[[ 0.3350,  0.7661,  2.0227,  ..., -0.3498, -0.7951, -0.7265],
          [ 0.4971,  1.7222,  1.2369,  ..., -0.7160, -0.8351, -0.7102],
          [-0.0191,  2.0960,  2.4046,  ..., -0.0222, -0.5633, -0.0805],
          ...,
          [ 0.7047,  0.4139,  1.2986,  ...,  0.1282, -0.5849, -0.7071],
          [ 1.2121,  0.6142,  0.3822,  ..., -0.1689, -0.2055, -0.1425],
          [ 0.4498, -0.0710,  0.3296,  ...,  0.8667,  0.3907,  0.3441]]]])


In [12]:
# Feel free to add more items here
config = {
    "lr"         : 1e-4,
    "epochs"     : 100,
    "batch_size" : 1,  # Increase if your device can handle it
    "num_classes": 1,
    'truncated_normal_mean' : 0,
    'truncated_normal_std' : 0.2,
}

In [13]:
# create a torch.utils.data.Dataset/DataLoader
annotation_json_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/annotation.json'
train_img_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/images'
train_mask_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/masks'

train_img_path_for_ImageFolder_dataloader = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/images_with_class/'

#! Temporarily using train and val images as same
val_img_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/images'
val_mask_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/masks'

test_img_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/images'

# img_size = (1384, 1032) # = width, height            # currently PtGrey images
img_size = (1024, 1024)

## Extract Masks from the COCO annotations (if not already done)

In [6]:
# convert_coco_to_mask(input_json=annotation_json_path, image_folder=train_img_path, output_folder=train_mask_path)

In [16]:
# Find mean and std of your dataset:
def get_mean_and_std_calculated(IMAGE_DATA_DIR):
    """
    NOTE: The ImageFolder dataloader requires the following file structure:

    root
    |
    └── cat (class label)
        |
        ├──img_2.png
        └──img_1.png

    """
    train_dataset = ImageFolder(IMAGE_DATA_DIR, transform=torchvision.transforms.ToTensor())

    # Initialize lists to store channel-wise means and standard deviations
    channel_wise_means = [0.0, 0.0, 0.0]
    channel_wise_stds = [0.0, 0.0, 0.0]

    # Iterate through the training dataset to calculate means and standard deviations
    for image, _ in train_dataset:
        for i in range(3):  # Assuming RGB images
            channel_wise_means[i] += image[i, :, :].mean().item()
            channel_wise_stds[i] += image[i, :, :].std().item()

    # Calculate the mean and standard deviation for each channel
    num_samples = len(train_dataset)
    channel_wise_means = [mean / num_samples for mean in channel_wise_means]
    channel_wise_stds = [std / num_samples for std in channel_wise_stds]

    # Print the mean and standard deviation for each channel
    print("Mean:", channel_wise_means)
    print("Std:", channel_wise_stds)

    return channel_wise_means, channel_wise_stds

# means, stds = get_mean_and_std_calculated(train_img_path_for_ImageFolder_dataloader)
means = [0.44895144719250346, 0.4951483853617493, 0.4498602793532975]
stds = [0.21388493326245522, 0.24571933703763144, 0.22413276759337405]

normalize_transform = torchvision.transforms.Compose([
    torchvision.transforms.Normalize(mean=means, std=stds) # always normalize only after tensor conversion
    ])

In [17]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(0.5),
    torchvision.transforms.ColorJitter(brightness=0.16, contrast=0.15, saturation=0.1),
    torchvision.transforms.RandomRotation(18),
    torchvision.transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    torchvision.transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
])

train_dataset = SemanticSegmentationDataset(img_paths=train_img_path, mask_paths=train_mask_path, size=img_size, mode='binary', normalize=normalize_transform, transformations=train_transforms)
val_dataset = SemanticSegmentationDataset(img_paths=val_img_path, mask_paths=val_mask_path, size=img_size, mode='binary', normalize=normalize_transform, transformations=None)
test_dataset = SemanticSegmentationDataset(img_paths=val_img_path, mask_paths=None, size=img_size, normalize=normalize_transform, transformations=None)

temp = train_dataset.__getitem__(1)

# Create data loaders
train_loader = DataLoader(
    dataset     = train_dataset,
    batch_size  = config['batch_size'],
    shuffle     = True,
    num_workers = 4,
    pin_memory  = True
)

val_loader = DataLoader(
    dataset     = val_dataset,
    batch_size  = config['batch_size'],
    shuffle     = False,
    num_workers = 2
)

test_loader = DataLoader(
    dataset     = test_dataset,
    batch_size  = config['batch_size'],
    shuffle     = False,
    drop_last   = False,
    num_workers = 2)

In [18]:
model = Unet(
    # backbone='convnext_base', # backbone network name
    backbone='resnet50',
    preprocessing=True,
    in_channels=3, # input channels (1 for gray-scale images, 3 for RGB, etc.)
    num_classes=config["num_classes"],  # output channels (number of classes in your dataset)
    encoder_freeze=True,
    pretrained=True,
)

# model = model().to(device)
random_tensor = torch.rand((1, 3, 1024, 1024))
print(model.predict(random_tensor))

tensor([[[[-1.1915, -1.0216, -1.3952,  ..., -2.3227, -2.3106,  0.1408],
          [-1.3971, -1.5565, -3.6995,  ..., -2.3611,  0.4789,  0.4479],
          [-0.9319,  0.0875, -2.5249,  ..., -3.3560, -2.0224, -2.7261],
          ...,
          [-1.5556, -2.4516, -2.1156,  ..., -2.1118, -1.7232, -2.7438],
          [-2.6119, -3.2381, -3.3083,  ..., -1.2819,  0.1450, -3.3783],
          [-2.4591, -2.1086, -3.5836,  ..., -1.4152, -0.9050, -3.5213]]]])


In [21]:
# Define wandb credentials

import wandb
wandb.login(key="49efd84d0e342f343fb91401332234dea4a3ffe2") #API Key is in your wandb account, under settings (wandb.ai/settings)

run = wandb.init(
    name = "UNet_with_resnet_50", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "IDL_Project_Segmentation", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: sushantj. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/sush/.netrc


In [22]:
checkpoint_path = '/home/sush/klab2/Segmentation_Models/checkpoints/checkpoint.pth'

In [23]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=config['lr'], betas=(0.9, 0.999), weight_decay=0.05)
gamma = 0.8
milestones = [10,20,40,60,80]

# scheduler1 = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=0.9, total_iters=5)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)
# scheduler3 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
# scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2, scheduler3], milestones=[20, 51])

mixed_precision_scaler = torch.cuda.amp.GradScaler()

trainer = Trainer(
    model=model,              # UNet model with Resnet50 backbone
    criterion=DiceLoss(),     # loss function
    optimizer=optimizer,
    epochs=10,
    # scaler=mixed_precision_scaler,
    lr_scheduler=scheduler,
    device=device,
    checkpoint_path=checkpoint_path
)

trainer.fit(train_loader, val_loader)

Validation: 100%|██████████| 98/98 [00:11<00:00,  8.49 validating-batch/s, loss=0.996]


Val Loss 0.9963
Saving model


Traning Model on 10 epochs:  10%|█         | 1/10 [00:57<08:36, 57.37s/it]


KeyboardInterrupt: 

In [17]:
# Check if the checkpoint file exists
if os.path.exists(checkpoint_path):
    # If the checkpoint file exists, load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']  # last epoch
    val_loss = checkpoint['val_loss']  # Update the best accuracy
    # Load the checkpoint and update the scheduler state if it exists in the checkpoint
    if 'scheduler_state_dict' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        print("Loaded scheduler state from checkpoint.")
    else:
        print("No scheduler state found in checkpoint.")
    print("Loaded checkpoint from:", checkpoint_path)
else:
    # If the checkpoint file does not exist, start training from scratch
    start_epoch = 0
    print("No checkpoint found at:", checkpoint_path)

print(model)

Loaded scheduler state from checkpoint.
Loaded checkpoint from: /home/sush/klab2/Segmentation_Models/checkpoints/checkpoint.pth
Unet(
  (encoder): FeatureListNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Ident